In [17]:
%run -i ../../src/reload.py
import src

In [18]:
%run ../../scripts/s8_make_datasets.py

Reading state logs...
Join contracts logs with transactions...
Checking, that every art house log entry has corresponding nft log entry...
Getting banned addrs and objkts...


In [19]:
print('Initializing datasets...')

tokens_ds = init_tokens_ds()
addrs_ds = init_addrs_ds()
swaps_ds = init_swaps_ds()
sells_ds = {}
transfers_ds = {}


def process_mint(tr_index_entry):
    token_id = nft_log_entry['token_id']
    token_count = nft_log_entry['count']
    tokens_receiver = nft_log_entry['tokens_receiver']

    meta_creator = created_token_db_entry['meta_creator']
    assert created_token_db_entry['state'] == 'NOT_EXISTS'
    created_token_db_entry['state'] = 'CREATED'
    assert created_token_db_entry['own_counts_by_address'] == None
    created_token_db_entry['own_counts_by_address'] = Counter({
        tokens_receiver: token_count,
    })

    author = ah_log_entry['mint_sender']
    assert ah_log_entry['count'] == nft_log_entry['count']
    assert ah_log_entry['tokens_receiver'] == nft_log_entry['tokens_receiver']
    assert ah_log_entry['tokens_receiver'] == created_token_db_entry['mint_tokens_receiver']
    assert ah_log_entry['method'] == 'apply_mint'

    addrs_ds[author]['mint_count'] += token_count
    
    receiver_category = get_addr_category(tokens_receiver, author)
    assert receiver_category in ['author', 'user']

    assert created_token_db_entry['own_counts_by_category'] == None
    created_token_db_entry['own_counts_by_category'] = Counter({
        receiver_category: token_count,
    })
    
    assert tr_volume == 0
    assert created_swap_db_entry is None
    assert money_log_entry is None

    transfers_ds[str(nft_log_entry['row_id'])] = {
        'tr': src.datasets.TrEvent(nft_log_entry['row_id']),
        'category': f'mint->{receiver_category}',
        'token_id': str(token_id),
        'price': 0,
        'count': token_count,
        'swap_id': '',
        'sender': '',
        'receiver': nft_log_entry['tokens_receiver'],
        'method': 'mint_OBJKT',
    }


def post_process():
    for token_db_entry in tokens_db.values():
        assert token_db_entry['state'] == 'CREATED'
        token_ds_entry = tokens_ds[str(token_db_entry['token_id'])]
        
        author_ds_entry = addrs_ds[token_ds_entry['issuer']]
        if author_ds_entry['ban_status'] == 'banned' and token_ds_entry['ban_status'] == '':
            token_ds_entry['ban_status'] = 'author_banned'
        if token_ds_entry['ban_status'] == 'banned' and author_ds_entry['ban_status'] == '':
            author_ds_entry['ban_status'] = 'some_tokens_banned'

        token_ds_entry['author_owns_count'] = token_db_entry['own_counts_by_category']['author']
        token_ds_entry['burn_count'] = token_db_entry['own_counts_by_category']['burn']
        token_ds_entry['other_own_count'] = token_db_entry['own_counts_by_category']['user']
        token_ds_entry['other_own_count'] += token_db_entry['own_counts_by_category']['ext']

        token_ds_entry['author_sent_count'] = max(0, token_db_entry['author_other_balance']) + \
            min(0, token_db_entry['author_swap_balance'])

        assert token_ds_entry['author_sent_count'] >= 0

        del token_ds_entry, token_db_entry

    for swap_db_entry in swaps_db.values():
        swap_ds_entry = swaps_ds[str(swap_db_entry['swap_id'])]
        token_ds_entry = tokens_ds[str(swap_db_entry['token_id'])]
        token_db_entry = tokens_db[str(swap_db_entry['token_id'])]
        author_ds_entry = addrs_ds[str(token_ds_entry['issuer'])]

        if swap_db_entry['state'] == 'ACTIVE':
            token_ds_entry['available_prices'].add(swap_ds_entry['price'], swap_ds_entry['available_count'])
            author_ds_entry['available_prices'].add(swap_ds_entry['price'], swap_ds_entry['available_count'])
        else:
            assert swap_db_entry['state'] == 'CLOSED', (swap_db_entry['state'], swap_db_entry['swap_id'])

        del token_ds_entry, token_db_entry

    for token_db_entry in tokens_db.values():
        token_ds_entry = tokens_ds[str(token_db_entry['token_id'])]

        assert token_ds_entry['mint_count'] == token_ds_entry['author_owns_count'] + \
            token_ds_entry['available_prices'].count('all') + token_ds_entry['other_own_count'] + \
            token_ds_entry['burn_count'], token_db_entry['token_id']

        assert token_ds_entry['author_sent_count'] <= token_ds_entry['other_own_count'] + \
            token_ds_entry['available_prices'].count('all'), token_db_entry['token_id']
        
        del token_ds_entry, token_db_entry

    # TODO: check assertions


for nft_log_entry in nft_state_log:
    nft_log_entry_method = nft_log_entry['method']
    assert nft_log_entry_method in ['apply_transfer', 'apply_mint'], nft_log_entry_method
    tr_index_entry = nft_log_entry['tr_index_entry']

    ah_log_entry = tr_index_entry['ah_log_entry']
    money_log_entry = tr_index_entry['money_log_entry']
    created_swap_db_entry = tr_index_entry['created_swap']
    created_token_db_entry = tr_index_entry['created_token']
    tr_info = tr_index_entry['tr_info']
    tr_volume = tr_info['volume']

    if nft_log_entry_method == 'apply_mint':
        process_mint(tr_index_entry)
        continue

    assert nft_log_entry_method == 'apply_transfer'

    for tx in nft_log_entry['txs']:
        assert created_token_db_entry is None

        tx_from = tx['from']
        tx_to = tx['to']
        tx_token_id = int(tx['token_id'])
        tx_count = int(tx['count'])

        if tx_count == 0:
            continue
        if tx_from == tx_to:
            continue

        token_db_entry = tokens_db[str(tx_token_id)]
        token_ds_entry = tokens_ds[str(tx_token_id)]
        author = token_ds_entry['issuer']
        author_db_entry = addrs_db[author]
        author_ds_entry = addrs_ds[author]
        
        sender_category = get_addr_category(tx_from, author)
        receiver_category = get_addr_category(tx_to, author)
        
        price = 0
        swap_id = ''
        method = ''

        if ah_log_entry and ah_log_entry['method'] == 'apply_cancel_swap':
            assert sender_category == 'swap'
            assert receiver_category in ['author', 'user']
            assert tr_volume == 0

            swap_id = str(ah_log_entry['swap_id'])
            
            swap_db_entry = swaps_db[str(ah_log_entry['swap_id'])]
            swap_db_entry['state'] = 'CLOSED'

            swap_ds_entry = swaps_ds[str(ah_log_entry['swap_id'])]
            swap_ds_entry['closed'].set_row_id(ah_log_entry['row_id'])
            
            swap_ds_entry['returned_count'] = swap_ds_entry['available_count']
            swap_ds_entry['available_count'] = 0

            del swap_db_entry, swap_ds_entry

        elif sender_category == 'swap':
            assert ah_log_entry['method'] == 'apply_collect'
            assert receiver_category in ['author', 'user']
            assert tr_volume >= 0

            swap_id = str(ah_log_entry['swap_id'])

            swap_ds_entry = swaps_ds[str(ah_log_entry['swap_id'])]
            swap_db_entry = swaps_db[str(ah_log_entry['swap_id'])]

            assert swap_db_entry['state'] == 'ACTIVE'
            assert created_swap_db_entry is None
            price_mutez = swap_db_entry['price']
            price = price_mutez / 1e6

            assert ah_log_entry['count'] == tx_count
            assert len(nft_log_entry['txs']) == 1

            if swap_db_entry['price'] == 0:
                assert money_log_entry is None
                money_log_entry= {
                    'royalties': 0,
                    'comission': 0,
                    'seller_income': 0,
                }

            else:
                assert money_log_entry
                assert money_log_entry['token_id'] == tx_token_id
                assert money_log_entry['token_count'] == tx_count

                assert abs(round(money_log_entry['price'] * 1e6) - price_mutez * tx_count) <= 1, (
                    round(money_log_entry['price'] * 1e6),
                    price_mutez,
                )
                assert money_log_entry['royalties_receiver'] == token_ds_entry['issuer']
                assert money_log_entry['payer'] == tx_to
                assert money_log_entry['seller'] == swap_ds_entry['created_by']

            by_author = int(swap_ds_entry['created_by'] == token_ds_entry['issuer'])

            sells_ds[str(tr_info['row_id'])] = {
                'tr': src.datasets.TrEvent(tr_info['row_id']),
                'token_id': str(tx_token_id),
                'count': tx_count,
                'seller': swap_ds_entry['created_by'],
                'buyer': tx_to,
                'price': price,
                'swap_id': swap_ds_entry['swap_id'],
                'by_author': by_author,
                'total_royalties': money_log_entry['royalties'],
                'total_comission': money_log_entry['comission'],
                'total_seller_income': money_log_entry['seller_income'],
            }

            if by_author:
                token_ds_entry['author_sold_prices'].add(price, tx_count)
                author_ds_entry['author_sold_prices'].add(price, tx_count)
            else:
                token_ds_entry['secondary_sold_prices'].add(price, tx_count)
                author_ds_entry['secondary_sold_prices'].add(price, tx_count)

            addrs_ds[tx_to]['bought_prices'].add(price, tx_count)
            swap_ds_entry['sold_count'] += tx_count
            swap_ds_entry['available_count'] -= tx_count
            swap_ds_entry['sold_price_sum'] += price * tx_count

            assert swap_ds_entry['available_count'] >= 0
            
            del swap_ds_entry, swap_db_entry, by_author

        else:
            assert sender_category in ['author', 'user', 'ext'], sender_category

            if receiver_category == 'swap':
                assert tr_volume == 0

                if not ah_log_entry:
                    # If token is sent to the Art House without calling contract
                    # it is equivalent to sending it to trash
                    receiver_category = 'burn'

                else:
                    assert ah_log_entry['method'] == 'apply_swap'
                    assert tr_volume == 0

                    swap_id = str(created_swap_db_entry['swap_id'])

                    swap_db_entry = swaps_db[str(created_swap_db_entry['swap_id'])]
                    assert swap_db_entry['state'] == 'NOT_EXISTS'
                    swap_db_entry['state'] = 'ACTIVE'

                    swap_ds_entry = swaps_ds[str(created_swap_db_entry['swap_id'])]
                    swap_ds_entry['available_count'] = swap_ds_entry['total_count']
                    del swap_db_entry, swap_ds_entry

            elif receiver_category == 'burn':
                assert tr_volume == 0
                assert ah_log_entry is None

            elif receiver_category == 'ext':
                assert tr_volume >= 0
                # we do not assign price in this case

            elif receiver_category in ['user', 'author']:
                assert ah_log_entry is None

                if sender_category == 'ext':
                    assert tr_volume >= 0
                    if tr_volume > 0:
                        assert money_log_entry
                        assert money_log_entry['method'] == 'apply_ext_swap'
                        price = money_log_entry['price'] / 1e6
                        method = money_log_entry['call']

                else:
                    assert tr_volume == 0

            else:
                assert False

        transfers_ds[str(nft_log_entry['row_id'])] = {
            'tr': src.datasets.TrEvent(nft_log_entry['row_id']),
            'category': f'{sender_category}->{receiver_category}',
            'token_id': str(tx_token_id),
            'price': price,
            'count': tx_count,
            'swap_id': swap_id,
            'sender': tx_from,
            'receiver': tx_to,
            'method': method,
        }

        token_db_entry['own_counts_by_category'][receiver_category] += tx_count
        token_db_entry['own_counts_by_category'][sender_category] -= tx_count
        assert token_db_entry['own_counts_by_category'][sender_category] >= 0

        token_db_entry['own_counts_by_address'][tx_to] += tx_count
        token_db_entry['own_counts_by_address'][tx_from] -= tx_count
        assert token_db_entry['own_counts_by_address'][tx_from] >= 0
        
        if sender_category == 'author':
            if receiver_category == 'swap':
                token_db_entry['author_swap_balance'] += tx_count
            elif receiver_category == 'burn':
                pass
            else:
                token_db_entry['author_other_balance'] += tx_count
        elif receiver_category == 'burn':
            token_db_entry['author_other_balance'] -= tx_count
        elif receiver_category == 'author':
            if sender_category == 'swap':
                token_db_entry['author_swap_balance'] -= tx_count
            else:
                token_db_entry['author_other_balance'] -= tx_count


post_process()

src.datasets.make_dataset(tokens_ds, config.dataset_dir / 'tokens', tr_info_db)
src.datasets.make_dataset(addrs_ds, config.dataset_dir / 'addrs', tr_info_db)
src.datasets.make_dataset(swaps_ds, config.dataset_dir / 'swaps', tr_info_db)
src.datasets.make_dataset(sells_ds, config.dataset_dir / 'sells', tr_info_db)
src.datasets.make_dataset(transfers_ds, config.dataset_dir / 'transfers', tr_info_db)

src.datasets.validate_datasets()

Initializing datasets...
written 37865540 bytes 24567 entries
written 6661773 bytes 4719 entries
written 14280191 bytes 30666 entries
written 31258982 bytes 79514 entries
written 49647014 bytes 144278 entries
Validating tokens ...

Validating addrs ...

Validating sells ...

Validating transfers ...

Validating swaps ...

